<a href="https://colab.research.google.com/github/ogigo/image_classification/blob/main/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
api_token = {"username":"shahedshoyab","key":"1aa2d9748604f09744dac84eb27b107f"}

import json

with open('/root/.kaggle/kaggle.json', 'w') as file:json.dump(api_token, file)

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download gpiosenka/100-bird-species

100% 1.69G/1.69G [01:20<00:00, 23.3MB/s]
100% 1.69G/1.69G [01:20<00:00, 22.6MB/s]


In [ ]:
!unzip 100-bird-species.zip

In [ ]:
import os
len(os.listdir("/content/train"))

450

In [ ]:
len(os.listdir("/content/test"))

450

In [ ]:
len(os.listdir("/content/valid"))

450

In [ ]:
import glob
names=glob.glob("/content/train/*")

In [ ]:
birds_name=[]
for name in names:
  birds_name.append(name.split("/")[-1])

In [ ]:
import numpy as np
class_names=np.array(sorted(birds_name))

In [ ]:
class_names

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential

In [ ]:
train_dir="/content/train/"
test_dir="/content/test/"
val_dir="/content/valid/"

In [ ]:
train_datagen=ImageDataGenerator(rescale=1./255)
test_datagen=ImageDataGenerator(rescale=1./255)
val_datagen=ImageDataGenerator(rescale=1./255)


In [ ]:
train_data=train_datagen.flow_from_directory(directory=train_dir,
                                             batch_size=32,
                                             target_size=(224,224),
                                             class_mode="categorical")

test_data=test_datagen.flow_from_directory(directory=test_dir,
                                             batch_size=32,
                                             target_size=(224,224),
                                             class_mode="categorical")

val_data=val_datagen.flow_from_directory(directory=val_dir,
                                             batch_size=32,
                                             target_size=(224,224),
                                             class_mode="categorical")

Found 70626 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.
Found 2250 images belonging to 450 classes.


In [ ]:
image=next(iter(train_data))

In [ ]:
image[0].shape

(32, 224, 224, 3)

In [ ]:
base_model=tf.keras.applications.InceptionV3(include_top=False)
base_model.trainable=False

87910968/87910968 [==============================] - 5s 0us/step


In [ ]:
inputs=tf.keras.layers.Input(shape=(224,2224,3),name="input_layer")
x=base_model(inputs)
x=tf.keras.layers.GlobalAveragePooling2D(name="pooling_layer")(x)
outputs=tf.keras.layers.Dense(450,activation="softmax",name="output_layer")(x)
model_01=tf.keras.Model(inputs,outputs)

In [ ]:
model_01.compile(loss = "categorical_crossentropy",
                optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01),
                metrics = ["accuracy"])

In [ ]:
history = model_01.fit(train_data,
                                 epochs=10,
                                 steps_per_epoch = len(train_data),
                                 validation_data = val_data,
                                 validation_steps = int(0.25*len(val_data)),)

Epoch 1/10
2208/2208 [==============================] - 202s 86ms/step - loss: 9.5702 - accuracy: 0.4908 - val_loss: 6.2685 - val_accuracy: 0.6857
Epoch 2/10
2208/2208 [==============================] - 187s 85ms/step - loss: 6.8013 - accuracy: 0.6829 - val_loss: 5.6079 - val_accuracy: 0.7316
Epoch 3/10
2208/2208 [==============================] - 184s 83ms/step - loss: 5.7856 - accuracy: 0.7404 - val_loss: 6.4845 - val_accuracy: 0.7463
Epoch 4/10
2208/2208 [==============================] - 185s 84ms/step - loss: 4.9814 - accuracy: 0.7828 - val_loss: 5.7228 - val_accuracy: 0.7812
Epoch 5/10
2208/2208 [==============================] - 184s 84ms/step - loss: 4.3643 - accuracy: 0.8090 - val_loss: 5.6942 - val_accuracy: 0.7721
Epoch 6/10
2208/2208 [==============================] - 186s 84ms/step - loss: 3.8986 - accuracy: 0.8305 - val_loss: 6.0166 - val_accuracy: 0.8051
Epoch 7/10
2208/2208 [==============================] - 185s 84ms/step - loss: 3.5907 - accuracy: 0.8478 - val_loss: 6

In [ ]:
model_01.evaluate(test_data)

71/71 [==============================] - 7s 94ms/step - loss: 5.2650 - accuracy: 0.8440


[5.265006065368652, 0.843999981880188]

In [ ]:
base_model.trainable=True
for layer in base_model.layers[:-10]:
  layer.trainable=False

In [ ]:
model_01.compile(loss="categorical_crossentropy",
                 optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                 metrics=["accuracy"])

In [ ]:
#for layer_number, layer in enumerate(model_01.layers[1].layers):
  #print(layer_number, layer.name, layer.trainable)

In [ ]:
initial_epochs = 10
fine_tune_epochs = initial_epochs + 1

# Refit the model
history_2 = model_01.fit(train_data,
                       epochs = fine_tune_epochs,
                       validation_data = val_data,
                       validation_steps = int(0.25*len(val_data)),
                       initial_epoch =  history.epoch[-1],) # Start the epoch where it left before


Epoch 10/11
2208/2208 [==============================] - 190s 84ms/step - loss: 0.4730 - accuracy: 0.9671 - val_loss: 2.6620 - val_accuracy: 0.9044
Epoch 11/11
2208/2208 [==============================] - 182s 82ms/step - loss: 0.2376 - accuracy: 0.9794 - val_loss: 2.6569 - val_accuracy: 0.8971


In [ ]:
model_01.evaluate(test_data)


71/71 [==============================] - 6s 84ms/step - loss: 2.4052 - accuracy: 0.9147


[2.405155897140503, 0.9146666526794434]